In [1]:
import numpy as np
import pandas as pd
from transformers import AutoTokenizer, T5ForConditionalGeneration
import torch

/Users/adamwagnerhoegh/miniconda3/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
file_path = "output/devset/devset_with_contexts.parquet.gzip"
with open(file_path, "rb") as f:
    dev_set = pd.read_parquet(f)

In [4]:
dev_set

,question,answer,text,pnumber,law number,tfidf_context,bm25_context,bert_cls_context,bert_max_context,bert_mean_context
0,"Hvad har ejeren af en ejerlejlighed, sammen me...","Grunden, fælles bestanddele og tilbehør.",Ejeren af en ejerlejlighed har sammen med andr...,3,LOV nr 908 af 18/06/2020,[Ejeren af en ejerlejlighed har sammen med and...,[Ejeren af en ejerlejlighed har sammen med and...,[Hver ejerlejlighed anses som en selvstændig f...,"[Det kan aftales, at lejeren af et blandet lej...",[Ved skriftlig budgivning opfordrer fogeden sk...
1,Hvem fastsætter eller aftaler bestemmelser om ...,Finansministeren fastsætter eller aftaler best...,Højskolen skal følge de af finansministeren fa...,30,LBK nr 780 af 08/08/2019,[Højskolen skal følge de af finansministeren f...,[Højskolen skal følge de af finansministeren f...,[Professionshøjskolens navn fastsættes i vedtæ...,[Virksomheden kan efter finansministerens nærm...,[Told- og skatteforvaltningen kan fastsætte re...
2,Hvad skal Beskæftigelsesministeriet og Finanst...,Den indsendte årsrapport skal i det mindste in...,Uden ugrundet ophold efter repræsentantskabets...,25 l,LBK nr 1110 af 10/10/2014,[Uden ugrundet ophold efter bestyrelsens godke...,[Uden ugrundet ophold efter bestyrelsens godke...,[Arbejdsmarkedets Erhvervssikring betaler afgi...,[Den godkendte årsrapport skal indsendes til E...,"[§ 10 finder ikke anvendelse for gaver, der er..."
3,Hvor mange procent må kapitalandele i og lån y...,Kapitalandele i og lån ydet til en virksomhed ...,Følgende grænser for Arbejdsmarkedets Tillægsp...,26 e,LBK nr 1110 af 10/10/2014,[Følgende grænser for Lønmodtagernes Dyrtidsfo...,[Følgende grænser for Lønmodtagernes Dyrtidsfo...,[Et pengeinstituts kapitalandele erhvervet for...,[Bestyrelsen i et e-pengeinstitut eller betali...,[Skadesforsikringsselskaberne skal efter hvert...
4,Hvad er en betingelse for retten til jobpræmie?,Det er en betingelse for retten til jobpræmie ...,"Det er en betingelse for retten til jobpræmie,...",9,LOV nr 287 af 29/03/2017,[Det er en betingelse for retten til jobpræmie...,[Det er en betingelse for retten til jobpræmie...,[Det er en betingelse for retten til jobpræmie...,[Beskæftigelsesministeren kan fastsætte budget...,"[Det påhviler anklagemyndigheden at påse, at e..."
...,...,...,...,...,...,...,...,...,...,...
98,Hvordan anføres kandidatlister på stemmesedler?,I særskilte felter.,Kandidatlisterne anføres på stemmesedlen i sær...,46,LBK nr 6 af 08/01/2024,[Valgbestyrelsen udarbejder stemmesedler på gr...,[Valgbestyrelsen udarbejder stemmesedler på gr...,[Kandidatlisterne anføres på stemmesedlen i sæ...,[Kulturministeren kan fastsætte nærmere regler...,"[Ministeren for fødevarer, landbrug og fiskeri..."
99,Hvem iværksætter beslaglæggelse?,Politiet.,Politiet iværksætter beslaglæggelse. Politiet ...,807,LBK nr 250 af 04/03/2024,"[Beløb, som en person har til gode hos en virk...","[Beløb, som en person har til gode hos en virk...","[Retsformanden leder forhandlingen., Arbejdsti...",[Studievalg Danmark kan drive indtægtsdækket v...,[Regionsrådet og kommunalbestyrelsen koordiner...
100,Hvis interesser skal foranstaltninger mod inte...,De forvaltede alternative investeringsfondes e...,En forvalter af alternative investeringsfonde ...,23,LBK nr 231 af 01/03/2024,[Delegation eller videredelegation af porteføl...,[Delegation eller videredelegation af porteføl...,"[Hvis en afdeling er bevisudstedende, skal den...",[Bestyrelsen i et e-pengeinstitut eller betali...,[Erhvervsstyrelsen kan foretage udgående kontr...
101,Hvad skal valgstyrere eller tilforordnede vælg...,At stemmekasserne er tomme.,Afstemningen begynder kl. Inden stemmeafgivnin...,38,LBK nr 1432 af 01/12/2023,[Afstemningen begynder kl. Inden stemmeafgivni...,[Afstemningen begynder kl. Inden stemmeafgivni...,"[Valgbestyrelsen underretter de valgte om, at ...",[Afstemningen begynder kl. Inden stemmeafgivni...,[Medmindre strengere straf er forskyldt efter ...


In [3]:
print(np.mean([len(item) for item in dev_set['bert_cls_context']]))

3.0


In [8]:
dev_set.loc[3, 'bert_cls_context']

array(['Et pengeinstituts kapitalandele erhvervet for puljemidler, hvor kunderne bærer risikoen, indgår ikke i opgørelsen af fradrag i kapitalgrundlaget.',
       'Priser for ydelser fra en netvirksomhed må ikke stige som følge af det vederlag, der erlægges ved overdragelse af virksomheder.',
       'Værdien af aktier, der i henhold til § 215 i lov om finansiel virksomhed overdrages til en forening, medregnes ikke ved opgørelse af foreningens skattepligtige indkomst.'],
      dtype=object)

In [ ]:
# load model and tokenizer and set device
model_name = ""  
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name)
DEVICE = "mps" if torch.backends.mps.is_available() else "cpu" # set up for mac here, change to cuda if needed
model.to(DEVICE)

/Users/adamwagnerhoegh/miniconda3/lib/python3.12/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


T5ForConditionalGeneration(
  (shared): Embedding(32128, 1024)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 1024)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=1024, out_features=1024, bias=False)
              (k): Linear(in_features=1024, out_features=1024, bias=False)
              (v): Linear(in_features=1024, out_features=1024, bias=False)
              (o): Linear(in_features=1024, out_features=1024, bias=False)
              (relative_attention_bias): Embedding(32, 16)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseActDense(
              (wi): Linear(in_features=1024, out_features=4096, bias=False)
              (wo): Linear(in_features=4096, out_features=1024, bias=False)
              (d

In [35]:
tokenizer.encode(' Spørgsmål:')

[3686, 48, 1]

In [5]:
cls_embeddings = torch.load('/Users/adamwagnerhoegh/ANLPDP_exam/output/embeddings/cls_embeddings_DanskBERT.pt')

In [17]:
cls_embeddings.size()

torch.Size([42593, 768])

In [15]:
torch.mean(torch.std(cls_embeddings, dim=1))

tensor(0.9152)

In [20]:
def random_dense_retrieval(question, embeddings, corpus, pooling='cls', k=3):
    """
    Function that takes a question and returns a list of paragraphs that are most relevant to the question
    pooling = 'cls', 'max' or 'mean'
    """
    
    embedding_vector = torch.rand(1, 768)

    # normalise embeddings (to get cosine similarity from dot product)
    embedding_vector_normalised = embedding_vector / torch.norm(embedding_vector, dim=-1, keepdim=True)
    embeddings_matrix_normalised = embeddings / torch.norm(embeddings, dim=-1, keepdim=True)

    # get paragraphs with highest similarity scores
    dense_retrieval = embeddings_matrix_normalised @ torch.transpose(embedding_vector_normalised, 0, 1)
    sorted_retrieval = torch.sort(dense_retrieval, descending=True, stable=True, dim=0)
    fixed_retrieval_list = [(item, idx) for (item, idx) in zip(sorted_retrieval[0], sorted_retrieval[1]) if torch.isnan(item) == False]
    top_k_indices = [item[1] for item in fixed_retrieval_list[:k]]
    document = [corpus[i]['text'] for i in top_k_indices]

    return document

In [ ]:
for question in dev_set['question']:
    random_dense_retrieval(question, )